# Tool for Time-Series Extraction

This tool requires a dataset of geotiff images as an input. Minor alterations will be required to code depending on naming conventions and extensions. See comments for details

## Import Packages

Tool has numerous required dependcies. Ensure all packages are installed before running.

In [5]:
# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import xarray as xr
from datetime import datetime, timedelta, date
from affine import Affine
from tqdm import tqdm

# For satellite data
import rasterio as rs
from rasterio.plot import show
import rioxarray
import osgeo
from osgeo import gdal

# Use of open_rasterio gives "DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray." 
# However, rioxarray does not have the transform information necessary to extract coordinates.
import warnings
warnings.filterwarnings('ignore')

## User Inputs

In [15]:
# Glacier name that appears in file name; this will change depending on your personal file naming conventions.
# Hemisphere glacier is located in. And the desired extraction point as latitude, longitude coordinates.

# Jakobshavn Isbrae Glacier in Greenland
# glacier, hemisphere, input_lat, input_lon = 'JAK', 'NORTH', 69.1119, -49.485 
glacier, hemisphere, input_lat, input_lon = 'JAK', 'NORTH',69.12257, -49.50090


# Pine Island Glacier in Antarctica
# glacier, hemisphere, input_lat, input_lon = 'PIG', 'SOUTH', -75.3, -99.1 
# glacier, hemisphere, input_lat, input_lon = 'PIG', 'SOUTH',-75.2546 , -98.9910

# Data naming info ----------> These will also need to be adjusted based on the users filename format.
file_extension = '.coffsN_mag_DuFil_yrF.gc.tiff'  
date_format = 'yymmdd'
date_interval_format = 'yymmdd_yymmdd'

## Extract all filenames

In [17]:
# Extract all filenames and create an array of all path names using glob module. 
# This will take a while if it is a large dataset. However, once loaded and saved it does not 
# need to be run again.

# This is the path to where your data is saved
data_path = "/Volumes/b0133/ee21lh/data/velocity/CPOM/"+ glacier + "/T*/*/*/*.coffsN_mag_DuFil_yrF.gc.tiff"

stack_paths = glob.glob(data_path)

np.save(glacier+'_'+str(input_lat)+'_'+str(input_lon)+'_paths', np.array(stack_paths)) # save paths as a numpy array

# Main Code

In [4]:
stack_paths = np.load(glacier+'_'+str(input_lat)+'_'+str(input_lon)+'_paths.npy') # load paths if not already

## Convert extraction cooridinates to WG84 

polar_convert package contains Python functions for converting polar stereographic coordinates. 

Adapted from: https://github.com/nsidc/polarstereo-lonlat-convert-py.

Original software was developed by the NASA National Snow and Ice Data Center Distributed Active Archive Center. Author: Chris Torrence, September 2019

polar_lonlat_to_xy() function was adapted for the northern hemisphere with a central meridian of -45 degrees to create a new function polar_lonlat_north_to_xy().

In [18]:
from polar_convert.constants import SOUTH
from polar_convert.constants import NORTH
from polar_convert import polar_lonlat_to_xy
from polar_convert import polar_lonlat_north_to_xy

true_scale_lat = 71  # true-scale latitude in degrees
re = 6378.137  # earth radius in km
e = 0.08181919 # earth eccentricity

# Convert input point to WGS 84 / Polar Stereographic
if hemisphere == 'NORTH':
    input_point = polar_lonlat_north_to_xy(input_lon, input_lat, true_scale_lat, re, e, hemisphere)
else:
    input_point = polar_lonlat_to_xy(input_lon, input_lat, true_scale_lat, re, e, hemisphere)

input_x,input_y = [input_point[0]*1000,input_point[1]*1000] # convert from km to m

## Extraction Function

For a large dataset this will take quite a while to run. But the print output will ensure it is running correctly.

In [22]:
first_date_index = -(len(file_extension)+len(date_interval_format))
second_date_index = -(len(file_extension)+len(date_format))

# array which data will be put in of form [date1,date2,speed]
time_series = np.empty([len(stack_paths),3]) 

for i in tqdm(np.arange(0,len(stack_paths))):
    
    # Extracting dates from file names and save to time_series
    date1 = stack_paths[i][first_date_index:first_date_index+len(date_format)]
    time_series[i,0] = date1
    date2 = stack_paths[i][second_date_index:second_date_index+len(date_format)] 
    time_series[i,1] = date2
    
    # Import Geotiff as a DataArray (da) object
    da = xr.open_rasterio(stack_paths[i])
    
    # Extract coordinates
    transform = Affine(*da.attrs["transform"])
    nx, ny = da.sizes["x"], da.sizes["y"]
    x, y = transform * np.meshgrid(np.arange(nx) + 0.5, np.arange(ny) + 0.5)
    
    # Define pandas dataframe
    d = {'x': x.flatten(), 'y': y.flatten(), 'speed (m/yr)': np.array(da).flatten()}
    df = pd.DataFrame(data=d)
    
    # Find point in dataset closest to the input latitude and longitude
    closest_x = df['x'].sub(input_x).abs().idxmin()
    closest_y = df['y'].sub(input_y).abs().idxmin()
    extraction_x,extraction_y = df['x'][closest_x], df['y'][closest_y]
    
    # Extract the speed data at the extraction point and save to time_series
    extract_data = df[(df['x']==extraction_x) & (df['y']==extraction_y)]
    time_series[i,2]=extract_data['speed (m/yr)'].values[0]
    
    
#     print('Processed ',i+1,' of ',len(stack_paths), 'images')

time_series_df = pd.DataFrame(time_series,columns = ['Date1','Date2','speed (m/yr)'])
time_series_df.to_csv(glacier+'_'+str(input_lat)+'_'+str(input_lon)+'_speed_time_series.csv',index=False)
                

100%|█████████████████████████████████████████| 681/681 [39:29<00:00,  3.48s/it]
